Đối tượng để crawl thông tin nhà bán: Website Batdongsan.com.vn
Cụ thể: Nhà riêng, Nhà biệt thự, liền kề, Nhà mặt phố, Shophouse, nhà phố thương mại.
Link: https://batdongsan.com.vn/ban-nha-dat-tp-hcm

# 1. Các cài đặt cần chuẩn bị

In [ ]:
!pip install selenium

In [ ]:
!pip install undetected-chromedriver

In [ ]:
"""
!pip install webdriver-manager
!pip install chromedriver-autoinstaller
"""

In [1]:
# Thư viện:
from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
import time
import undetected_chromedriver as uc
#import chromedriver_autoinstaller
import pandas as pd
import csv

# 2. Tạo file CSV chứa danh sách đường dẫn tới trang thông tin chi tiết của từng ngôi nhà được rao bán trên batdongsan.com

In [ ]:
#chromedriver_autoinstaller.install()
options = webdriver.ChromeOptions()
#driver = webdriver.Chrome()
driver = uc.Chrome()

# Trả về một List chứa các đường dẫn URL tới trang thông tin chi tiết của từng ngôi nhà được rao bán
def getOnePage_All_IndividualURL(link_elements):
    onePage_url_list = []
    for link in link_elements: # Duyệt qua từng element trong link_elements,
        href = link.get_attribute("href") # nếu thấy có đường dẫn tới URL của một ngôi nhà,
        onePage_url_list.append(href) # thì append chúng vào biến trả về.
        #print(href)
    #print("Total number of Houses in this page =", len(onePage_url_list))
    return onePage_url_list

# Trả về số trang cuối cùng (kiểu int) trong tìm kiếm bán nhà đất ở TP.HCM, ví dụ: 1251
def getLastPage_Number(NextPage_xpath):
    driver.get(f"https://batdongsan.com.vn/ban-nha-dat-tp-hcm")
    nextPage_list = driver.find_elements("xpath", NextPage_xpath)
    lastPage = int(nextPage_list[-1].text.replace(".", "")) # Lấy trang cuối cùng (cũng là elemnt cuối), convert từ string sang int.
    #print(type(lastPage))
    print("The last Page is: ", lastPage)
    return lastPage

individualURL_xpath = '//a[@class="js__product-link-for-product-id"]'# individualURL_xpath = '//a[@class="js__product-link-for-product-id"]/@href'
NextPage_xpath = '//a[@class="re__pagination-number"]'
lastPage = getLastPage_Number(NextPage_xpath)

links = [] # Tạo một biến List để chứa toàn bộ URL dẫn đến trang cá nhân chi tiết của tất cả các ngôi nhà.
for i in range(1, lastPage+1): # Gọi vòng lập để duyệt tất cả các Page
    temp_links = []
    driver.get(f"https://batdongsan.com.vn/ban-nha-dat-tp-hcm/p{i}") # Mở từng Page lên.
    link_elements = driver.find_elements("xpath", individualURL_xpath) # Lấy toàn bộ element có xpath giống với individualURL_xpath.
    temp_links = getOnePage_All_IndividualURL(link_elements)
    for e in temp_links: # Dùng vòng lập để hoàn thiện biến list chứa URL của tất cả ngôi nhà.
        links.append(e)

time.sleep(10) # Dừng lại 10 giây.
driver.close() # Đóng cửa sổ mà Selenium đang automated control.

In [8]:
driver.quit()
print("Total number of Houses find =", len(links))

Total number of Houses find = 29240


In [9]:
# Xóa bớt các url bị trùng lập:
print("Total number of Houses Original =", len(links))
links2 = [*set(links)]
print("Total number of Houses after removing duplicate: ", len(links2))

Total number of Houses Original = 29240
Total number of Houses after removing duplicate:  27862


In [10]:
# Xuất ra file CSV
diction = {'href': links2} # Thêm tên cột "href", nơi chứa các đường link bán nhà.
df0 = pd.DataFrame(diction) # Convert biến Dictionary thành Pandas Dataframe.
df0.to_csv('URLs_data.csv') # Lưu dataframe thành file CSV.

# 3. Duyệt từng page chi tiết, crawl toàn bộ thông tin của từng ngôi nhà rao bán

In [ ]:
# Chuẩn bị sẵn file output CSV rỗng, chỉ có tên các cột mà thôi
with open('dataset.csv', 'w', newline='', encoding="utf-8") as csvfile:
    # writerow() method = write a single row into a CSV file.
    # writerows() method = write multiple rows to a CSV file.
    writerObj = csv.writer(csvfile)
    fields = ["Tiêu đề", "Địa chỉ", "Khu vực", "Ngày đăng", "Ngày hết hạn", "Loại tin", "Mã tin", "Người đăng", "Khoảng giá phổ biến Min", "Giá phổ biến nhất", "Khoảng giá phổ biến Max", "Giá của tin đang xem", "Diện tích", "Mức giá", "Mặt tiền", "Đường vào", "Hướng nhà", "Hướng ban công", "Số tầng", "Số phòng ngủ", "Số toilet", "Pháp lý", "Nội thất"]
    writerObj.writerow(fields)

## 3.1 Nạp dữ liệu

In [2]:
# Load file CSV chứa các URL dẫn tới từng ngôi nhà đã hoàn thành ở phần trước
df1 = pd.read_csv('URLs_data.csv') # "URLs_data.csv" phải ở cùng thư mục với notebook này.
df1['href'] = df1['href'].astype(pd.StringDtype()) # Convert Object thành String.
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27862 entries, 0 to 27861
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  27862 non-null  int64 
 1   href        27862 non-null  string
dtypes: int64(1), string(1)
memory usage: 435.5 KB


In [14]:
# Tạo biến list chứa các url để duyệt từng cái một trong vòng lặp for
urls_list = df1['href'].values.tolist() 
print(urls_list[20499:20499+2])

['https://batdongsan.com.vn/ban-nha-biet-thu-lien-ke-duong-tran-ngoc-dien-phuong-thao-dien/ban-nhanh-quan-2-dt-10x20-ham-3-lau-ho-boi-san-vuon-gia-35-ty-pr36590138', 'https://batdongsan.com.vn/ban-nha-rieng-duong-vo-thi-nho-phuong-tan-thuan-dong-1/q7-hxh-kdc-nam-long-hiem-co-ban-dt-70m2-nhinh-6-ty-chu-nhiet-ban-gap-pr37494857']


## 3.2 Hàm tính toán và làm việc

In [4]:
# Write a Single Row to a CSV File from a Python Dictionary
def addingSingleRow_toBaseCSV(dictVarFinal):
    with open('dataset.csv', 'a+', newline='', encoding="utf-8") as csv_file:
        headers = ["Tiêu đề", "Địa chỉ", "Khu vực", "Ngày đăng", "Ngày hết hạn", "Loại tin", "Mã tin", "Người đăng", "Khoảng giá phổ biến Min", "Giá phổ biến nhất", "Khoảng giá phổ biến Max", "Giá của tin đang xem", "Diện tích", "Mức giá", "Mặt tiền", "Đường vào", "Hướng nhà", "Hướng ban công", "Số tầng", "Số phòng ngủ", "Số toilet", "Pháp lý", "Nội thất"]
        writerObj = csv.DictWriter(csv_file, fieldnames=headers, delimiter=',')
        writerObj.writerow(dictVarFinal)

In [5]:
# Nhận vào 2 biến List, trả về 1 biến Dictionary
def getMax11Fields_dependOnInputLength(titleList, valueList):
    dictVar = dict(zip(titleList, valueList)) # Convert 2 list into 1 dictionary.
    #print("Important Fields of this House: " + str(dictVar))
    return dictVar

# Nhận vào 2 biến Dictionary, cập nhật Dictionary thứ hai vào biến Dictionary thứ nhất, xóa Dictionary thứ hai
def Merge(dict1, dict2):
    return(dict1.update(dict2)) # returns None. Vì vậy không cần gán biến mới cho hàm làm gì.

# Gom tất cả Fields về thành một Dictionary
def uniteAll_Into1Dict(tieude, diachi, khuvuc, ngaydang, ngayhethan, loaitin, matin, nguoidang, khoanggiaphobien_min, khoanggiaphobien_popular, khoanggiaphobien_max, khoanggiaphobien_ofThisHouse, dictVar):
    dictVarFinal = {"Tiêu đề":tieude, 
                    "Địa chỉ":diachi, 
                    "Khu vực":khuvuc, 
                    "Ngày đăng":ngaydang,
                    "Ngày hết hạn":ngayhethan, 
                    "Loại tin":loaitin, 
                    "Mã tin":matin, 
                    "Người đăng": nguoidang,
                    "Khoảng giá phổ biến Min": khoanggiaphobien_min,
                    "Giá phổ biến nhất": khoanggiaphobien_popular,
                    "Khoảng giá phổ biến Max": khoanggiaphobien_max,
                    "Giá của tin đang xem": khoanggiaphobien_ofThisHouse}
    Merge(dictVarFinal, dictVar)
    return dictVarFinal

In [6]:
# Chạy undetected chromedriver để lấy dữ liệu có trong link url đưa vào
def scrapAll_fromSingleURL(driver, individual_HouseUrl):
    #driver = uc.Chrome()
    #driver.maximize_window()
    driver.get(individual_HouseUrl)
    
    time.sleep(5) # Chờ trang load xong.
    # PART-I: Đơn giản, ít bị lỗi
    tieude = driver.find_elements("xpath", '//h1[@class="re__pr-title pr-title js__pr-title"]')
    tieude = tieude[0].text
    diachi = driver.find_elements("xpath", '//span[@class="re__pr-short-description js__pr-address"]')
    diachi = diachi[0].text
    category = driver.find_elements("xpath", '//a[@class="re__link-se"]')
    khuvuc = category[2].text

    # PART-II: Đơn giản, ít bị lỗi
    ngayloaima = driver.find_elements("xpath", '//div[@class="re__pr-short-info-item js__pr-config-item"]//span[@class="value"]')
    #print(len(ngayloaima))
    ngaydang = ngayloaima[0].get_attribute("innerHTML")
    ngayhethan = ngayloaima[1].get_attribute("innerHTML")
    loaitin = ngayloaima[2].get_attribute("innerHTML")
    matin = ngayloaima[3].get_attribute("innerHTML")

    # PART-III: Có thể gặp lỗi
    nguoidang_raw = driver.find_elements("xpath", '//div[@class="re__contact-name js_contact-name"]//a[@tracking-id="user-listing-ldp"]')[0].get_attribute("innerHTML")
    #print(len(nguoidang_raw))
    nguoidang = nguoidang_raw.strip() # Xóa các khoảng trắng, ví dụ: "   Nguyễn Văn A   ".

    # PART-IV: Gồm những data quan trọng nhất
    item_title_raw = driver.find_elements("xpath", '//div[@class="re__pr-specs-content-item"]//span[@class="re__pr-specs-content-item-title"]')
    item_value_raw = driver.find_elements("xpath", '//div[@class="re__pr-specs-content-item"]//span[@class="re__pr-specs-content-item-value"]')
    titleList = [] # Tại trang batdongsan.com.vn thì phần này có trình bày thị giác giống với bảng
    valueList = [] # Ví dụ: Mặt tiền | 4 m | Đường vào | 6 m
    for index in range(0, len(item_title_raw)):
        titleList.append(item_title_raw[index].get_attribute("innerHTML"))
        valueList.append(item_value_raw[index].get_attribute("innerHTML"))
    dictVar = getMax11Fields_dependOnInputLength(titleList, valueList)
    
    # PART-V: Có thể gặp lỗi
    element = driver.find_element("xpath", '//h3[@class="re__section-title"]')
    driver.execute_script("arguments[0].scrollIntoView();", element) # Lăn màn hình xuống chỗ có dòng chữ "Khoảng giá phổ biến".
    time.sleep(5) # và chờ 5 giây để trang load các text là giá trị n trăm triệu trên mét vuông.
    khoanggiaphobien_path = '//div[@class="pricing-content"]//div[@class="pricing-content-row"]//div[@class="pricing-content--r2"]'
    khoanggiaphobien = driver.find_elements("xpath", khoanggiaphobien_path) # Lần lượt chứa Min, nổi bật nhất, Max trong khu vực.
    khoanggiaphobien_min = khoanggiaphobien[0].text
    khoanggiaphobien_popular = khoanggiaphobien[1].text
    khoanggiaphobien_max = khoanggiaphobien[2].text
    khoanggiaphobien_ofThisHouse = driver.find_elements("xpath", '//div[@class="pricing-product"]//div[@class="pricing-content"]//div[@class="pricing-content--r2"]')
    khoanggiaphobien_ofThisHouse = khoanggiaphobien_ofThisHouse[0].text # Giá bán n trăm triệu trên mét vuông của tin đang xem.
    '''
    # PART-VI: Rất dễ xảy ra lỗi, lỗi thường xuyên
    buttonPath = '//div[@class="re__btn re__btn-cyan-solid--md phone js__phone"]' # Thẻ Div chứa số điện thoại đang bị ẩn ***, 
    # lưu ý là nên thu nhỏ Developer tools lại, để thấy class của button này khi nó đứng thẳng (phóng to DevTools làm nó nằm ngang và địa chỉ class bị ẩn). 
    wait = WebDriverWait(driver, 10)
    button = driver.find_element("xpath", buttonPath)
    button.click()
    wait = WebDriverWait(driver, 30) # nếu đợi <10 giây thì nó sẽ lỗi, bởi vì SĐT vẫn còn bị ẩn ***.
    sdt_raw1 = driver.find_elements("xpath", '//span[@class="re__content"]//span')
    sdt_raw2 = sdt_raw1[0].get_attribute("innerHTML")
    sdt = sdt_raw2.replace(" ", "")
    time.sleep(40)
    '''
    dictVarFinal = uniteAll_Into1Dict(tieude, diachi, khuvuc, ngaydang, ngayhethan, loaitin, matin, nguoidang, khoanggiaphobien_min, khoanggiaphobien_popular, khoanggiaphobien_max, khoanggiaphobien_ofThisHouse, dictVar)
    #driver.close()
    return dictVarFinal

## 3.3 Chạy crawl

In [7]:
# Nếu gặp phải trường hợp một trang tin bán nhà nào đó không crawl về được,
# thì crawl url lưu sẵn trong biến "write_thisWhenError" này để thay thế.
# Điều này giúp đảm bảo index của urls_list không bị xáo trộn,
# Tuy nhiên, nó cũng đồng nghĩa với việc tạo duplicate.
write_thisWhenError = urls_list[2] # URL tại vị trí thứ 3 trong danh sách.
print("Variable type =", type(write_thisWhenError)) # Chú ý, đường dẫn phải luôn là kiểu string.
print(write_thisWhenError)

Variable type = <class 'str'>
https://batdongsan.com.vn/ban-nha-mat-pho-duong-so-8-phuong-linh-chieu/-moi-dep-xay-kien-co-1t3l-tien-kd-vp-gan-vincom-thu-duc-ngang-6m-vuong-vuc-san-rong-oto-pr36835594


In [11]:
# MAIN
driver = uc.Chrome(version_main=114)
driver.maximize_window() # Phóng to màn hình.

# Tips: Chỉnh lại "urls_list[index_start:index_end+1]" để crawl từng phần nhỏ, thay vì làm tất cả trong một lần.
# Nên thử dần dần xem có lỗi gì hay không, ví dụ [:5], [:100] rồi hẵng lên đến vài nghìn mẫu dữ liệu. Nhằm tránh sửa đi sửa lại, giữa chừng bị ngắt chương trình.
for EachHouseURL in urls_list[20500:23000]:
    try: # Với từng url của trang tin bán nhà chi tiết, chúng ta sẽ crawl về những thông tin tương ứng
        dictVarFinal = scrapAll_fromSingleURL(driver, EachHouseURL) # Tổng hợp chúng lại thành một biến Dictionary
        addingSingleRow_toBaseCSV(dictVarFinal) # Sau đó, viết chúng ra thành một dòng trong file dataset.csv
    except: # Nếu xảy ra lỗi gì đó, thì crawl url chuẩn bị sẵn thay thế
        dictVarFinal = scrapAll_fromSingleURL(driver, write_thisWhenError)
        addingSingleRow_toBaseCSV(dictVarFinal)

driver.close()
driver.quit()

In [ ]:
# Dùng để test cơ bản khi chạy với ít dữ liệu.
# In ra mẫu vừa được crawl gần nhất:
#print(len(dictVarFinal))
#print(dictVarFinal)

In [ ]:
# Lệnh để tắt máy sau khi crawl xong cell trên:
#import os
#os.system("shutdown /s /t 1")

## 3.4 Xem lại Dataset

In [12]:
df2 = pd.read_csv('dataset.csv')
print("Độ dài dataset là: ", len(df2))
df2.info()

Độ dài dataset là:  20500
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20500 entries, 0 to 20499
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Tiêu đề                      20498 non-null  object
 1   Địa chỉ                      20498 non-null  object
 2   Khu vực                      20499 non-null  object
 3   Ngày đăng                    20500 non-null  object
 4   Ngày hết hạn                 20500 non-null  object
 5   Loại tin                     20500 non-null  object
 6   Mã tin                       20500 non-null  int64 
 7   Người đăng                   20500 non-null  object
 8   Khoảng giá phổ biến Min  19078 non-null  object
 9   Giá phổ biến nhất            18889 non-null  object
 10  Khoảng giá phổ biến Max  20185 non-null  object
 11  Giá của tin đang xem         18123 non-null  object
 12  Diện tích                    20498 non-null  object
 13  Mức g

In [ ]:
df2[0:4].filter(items=["Người đăng", "Khoảng giá phổ biến Min", "Giá phổ biến nhất", "Khoảng giá phổ biến Max", "Giá của tin đang xem", "Diện tích", "Mức giá"])

In [13]:
df2.tail(3)

,Tiêu đề,Địa chỉ,Khu vực,Ngày đăng,Ngày hết hạn,Loại tin,Mã tin,Người đăng,Khoảng giá phổ biến Min,Giá phổ biến nhất,...,Mức giá,Mặt tiền,Đường vào,Hướng nhà,Hướng ban công,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất
20497,"Villa sân vườn đường Lê Văn Sỹ, DT: 7x18m. Giá...","Đường Lê Văn Sỹ, Phường 12, Quận 3, Hồ Chí Minh",Quận 3,13/06/2023,23/06/2023,Tin thường,36732299,Văn Lai,144 triệu/m²,192 triệu/m²,...,"17,5 tỷ",NaN,NaN,NaN,NaN,3 tầng,4 phòng,5 phòng,Sổ đỏ/ Sổ hồng,NaN
20498,"Vip P. 10, Q. Phú Nhuận, sát MT, thông Nguyễn ...","Phường 10, Phú Nhuận, Hồ Chí Minh",Phú Nhuận,16/06/2023,26/06/2023,Tin thường,37469613,Nguyễn Ngọc Trúc Mai,173 triệu/m²,221 triệu/m²,...,"16,6 tỷ",NaN,NaN,NaN,NaN,4 tầng,5 phòng,7 phòng,Sổ đỏ/ Sổ hồng,Đầy đủ
20499,Bán nhanh biệt thự Thảo Điền Quận 2 - DT 10x20...,"Đường Trần Ngọc Diện, Phường Thảo Điền, Quận 2...",Quận 2,15/06/2023,25/06/2023,Tin thường,36590138,Nhật Nam,166 triệu/m²,236 triệu/m²,...,35 tỷ,10 m,8 m,NaN,NaN,4 tầng,5 phòng,5 phòng,Sổ đỏ/ Sổ hồng,Đầy đủ
